In [1]:
# Use Google Colab
use_colab = True
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/685'

    !pip install -r requirements.txt

!pip install sentencepiece

from datasets import load_dataset

import sys
import os
import random
import shutil
import copy
import inspect


import numpy as np
import torch
import transformers
import datasets
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tqdm


Dataset preproccesing script


In [3]:
import pandas as pd

# text_df_a = pd.read_csv("./OLIDv1/testset-levela.tsv", sep="\t")
# labels_df_a = pd.read_csv("./OLIDv1/labels-levela.csv", sep=",")
# merged_df_a = pd.merge(text_df_a, labels_df_a, on='id', how='outer')
# merged_df_a.to_csv("olid_test_subtask_a.csv",sep="\t",index=False)

# text_df_b = pd.read_csv("./OLIDv1/testset-levelb.tsv", sep="\t")
# labels_df_b = pd.read_csv("./OLIDv1/labels-levelb.csv", sep=",")
# merged_df_b = pd.merge(text_df_b, labels_df_b, on='id', how='outer')
# merged_df_b.to_csv("olid_test_subtask_b.csv",sep="\t",index=False)

# text_df_c = pd.read_csv("./OLIDv1/testset-levelc.tsv", sep="\t")
# labels_df_c = pd.read_csv("./OLIDv1/labels-levelc.csv", sep=",")
# merged_df_c = pd.merge(text_df_c, labels_df_c, on='id', how='outer')
# merged_df_c.to_csv("olid_test_subtask_c.csv",sep="\t",index=False)


# train_df = pd.read_csv("./OLIDv1/olid-training-v1.0.tsv", sep="\t").fillna('not present')
# train_df.to_csv("olid_train_v2.csv",sep="\t",index=False)

In [4]:


from datasets import load_dataset
train_dataset = load_dataset('csv', data_files={'train': 'olid_train_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_a_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_a.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_b_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_b.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_c_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_c.csv'}, delimiter="\t", cache_dir='./olid_cache')

Using custom data configuration default-83328cf3ab9020af
Reusing dataset csv (./olid_cache/csv/default-83328cf3ab9020af/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-aab40cbe22b0ae62
Reusing dataset csv (./olid_cache/csv/default-aab40cbe22b0ae62/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-4c126f2099f7d8bf
Reusing dataset csv (./olid_cache/csv/default-4c126f2099f7d8bf/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-faf03cde57e11c2d
Reusing dataset csv (./olid_cache/csv/default-faf03cde57e11c2d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
import torch

import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(685)

tokenizer = T5Tokenizer.from_pretrained('t5-small', cache_dir='./t5_cache')
model = T5ForConditionalGeneration.from_pretrained('t5-small', cache_dir='./t5_cache')

train_dataloader = torch.utils.data.DataLoader(train_dataset['train'], shuffle=False, batch_size=8)


# the following 2 hyperparameters are task-specific
max_source_length = 512
max_target_length = 30
training_epochs = 5

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

In [ ]:
for i in train_dataset['train']:
  print(i)
  break

{'id': 86426, 'tweet': '@USER She should ask a few native Americans what their take on this is.', 'subtask_a': 'OFF', 'subtask_b': 'UNT', 'subtask_c': 'not present'}


In [4]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

In [9]:
#model.load_state_dict(torch.load(os.path.join('./olid_trained_models/', 't5_full_shot_epoch_1.pt')))

training_epochs = 5
start_epoch = 0
for epoch in tqdm.notebook.tqdm(range(start_epoch,training_epochs)):
  print("Training epoch %d" % epoch)
  print()
  model.train()

  for batch in tqdm.notebook.tqdm(train_dataloader):
    # text = batch['text']
    input_sequences=[]
    output_sequences = []
    for idx, text in enumerate(batch['tweet']):
      if batch['subtask_a'][idx] == "NOT":
        input_sequences.append(text + '<\s> ' + 'Offensive speech is ')
        output_sequences.append("not present.")
      else:
        input_sequences.append(text + '<\s> ' + 'Offensive speech is ')
        output_sequences.append("present.")
        if batch['subtask_b'][idx] == "UNT":
          input_sequences.append(text + '<\s> ' + 'Targeted offensive speech is ')
          output_sequences.append("not present.")
        else:
          input_sequences.append(text + '<\s> ' + 'Targeted offensive speech is ')
          output_sequences.append("present.")
          if batch['subtask_c'][idx] == "IND":
            input_sequences.append(text + '<\s> ' + 'The offensive speech is targeted towards ')
            output_sequences.append("individual.")
          elif batch['subtask_c'][idx] == "GPR":
            input_sequences.append(text + '<\s> ' + 'The offensive speech is targeted towards ')
            output_sequences.append("group.")
          else:
            input_sequences.append(text + '<\s> ' + 'The offensive speech is targeted towards ')
            output_sequences.append("neither.")

    # encode the inputs
    
    encoding = tokenizer(input_sequences,
                        padding='longest',
                        max_length=max_source_length,
                        truncation=True,
                        return_tensors="pt")
    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

    # encode the targets
    target_encoding = tokenizer(output_sequences,
                                padding='longest',
                                max_length=max_target_length,
                                truncation=True)
    labels = target_encoding.input_ids

    # replace padding token id's of the labels by -100
    labels = [
              [(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels
    ]
    labels = torch.tensor(labels)

    # forward pass
    loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  torch.save(model.state_dict(),os.path.join('./olid_trained_models/', 't5_full_shot_epoch_{}.pt'.format(epoch)))

  0%|          | 0/3 [00:00<?, ?it/s]

Training epoch 2



  0%|          | 0/1655 [00:00<?, ?it/s]

Training epoch 3



  0%|          | 0/1655 [00:00<?, ?it/s]

Training epoch 4



  0%|          | 0/1655 [00:00<?, ?it/s]

subtask_a evaluation

In [8]:
test_a_dataset

#NOT = 620
#OFF = 240

DatasetDict({
    test: Dataset({
        features: ['id', 'tweet', 'label'],
        num_rows: 860
    })
})

In [16]:
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], batch_size=8)

with torch.no_grad():

  model.load_state_dict(torch.load(os.path.join('./olid_trained_models/', 't5_full_shot_epoch_4.pt')))

  model.eval()
  count_1 = 0
  count_2 = 0

  for batch in tqdm.notebook.tqdm(test_dataloader):
    # when generating, we will use the logits of right-most token to predict the next token
    # so the padding should be on the left
    tokenizer = T5Tokenizer.from_pretrained('t5-small', cache_dir='./t5_cache')

    sentences = batch['tweet'] # use different length sentences to test batching
    for input in sentences:
      input = input + '<\s> ' + 'Offensive speech is '
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)

    output_sequences = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=64,
        early_stopping=True,
        num_beams=10,
        num_return_sequences=1,
        no_repeat_ngram_size=2
    )
    preds = (tokenizer.batch_decode(output_sequences, skip_special_tokens=True,clean_up_tokenization_spaces=True))
    print (preds)
    for i,pred in enumerate(preds):
      if "present." == pred and batch['label'][i]=='OFF':
        count_1 = count_1 + 1
      if "not present."==pred and (batch['label'][i]=='NOT'):
        count_2 = count_2 + 1
    print(count_1, count_2)


  0%|          | 0/108 [00:00<?, ?it/s]

['not present.', 'not present.', 'not present.', 'not present.', 'not present.', 'present.', 'not present.', 'not present.']
1 4
['not present.', 'not present.', 'not present.', 'not present.', 'not present.', 'not present.', 'present.', 'present.']
3 10
['not present.', 'not present.', 'not present.', 'present.', 'not present.', 'not present.', 'not present.', 'not present.']
3 17
['not present.', 'present.', 'not present.', 'not present.', 'not present.', 'present.', 'not present.', 'not present.']
4 21
['not present.', 'not present.', 'not present.', 'not present.', 'not present.', 'not present.', 'present.', 'present.']
6 27
['not present.', 'present.', 'present.', 'not present.', 'not present.', 'not present.', 'not present.', 'present.']
7 30
['present.', 'not present.', 'not present.', 'not present.', 'not present.', 'not present.', 'not present.', 'not present.']
8 36
['not present.', 'not present.', 'present.', 'present.', 'present.', 'not present.', 'not present.', 'present.'

In [ ]:
Subtask a results:
# 710/860 = 0.82 (Overall accuracy)
#145/240 = 0.6 (Accuracy on OFF)
#565/620 = 0.91 (Accuract on NOT)


In [17]:
test_b_dataset['test']

Dataset({
    features: ['id', 'tweet', 'label'],
    num_rows: 240
})

In [18]:
test_dataloader = torch.utils.data.DataLoader(test_b_dataset['test'], batch_size=8)

with torch.no_grad():

  model.load_state_dict(torch.load(os.path.join('./olid_trained_models/', 't5_full_shot_epoch_4.pt')))

  model.eval()
  count_1 = 0
  count_2 = 0

  for batch in tqdm.notebook.tqdm(test_dataloader):
    # when generating, we will use the logits of right-most token to predict the next token
    # so the padding should be on the left
    tokenizer = T5Tokenizer.from_pretrained('t5-small', cache_dir='./t5_cache')

    sentences = batch['tweet'] # use different length sentences to test batching
    for input in sentences:
      input = input + '<\s> ' + 'Targeted offensive speech is '
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)

    output_sequences = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=64,
        early_stopping=True,
        num_beams=10,
        num_return_sequences=1,
        no_repeat_ngram_size=2
    )
    preds = (tokenizer.batch_decode(output_sequences, skip_special_tokens=True,clean_up_tokenization_spaces=True))
    print (preds)
    for i,pred in enumerate(preds):
      if "present." == pred and batch['label'][i]=='TIN':
        count_1 = count_1 + 1
      if "not present."==pred and (batch['label'][i]=='UNT'):
        count_2 = count_2 + 1
    print(count_1, count_2)


  0%|          | 0/30 [00:00<?, ?it/s]

['not present.', 'not present.', 'present.', 'not present.', 'present.', 'present.', 'not present.', 'present.']
3 0
['not present.', 'present.', 'present.', 'not present.', 'not present.', 'present.', 'present.', 'not present.']
6 1
['present.', 'not present.', 'present.', 'present.', 'present.', 'present.', 'not present.', 'not present.']
10 1
['present.', 'present.', 'present.', 'present.', 'not present.', 'not present.', 'present.', 'not present.']
15 1
['not present.', 'present.', 'not present.', 'present.', 'not present.', 'present.', 'not present.', 'not present.']
18 1
['not present.', 'present.', 'present.', 'not present.', 'not present.', 'not present.', 'present.', 'present.']
21 1
['present.', 'not present.', 'not present.', 'not present.', 'not present.', 'not present.', 'present.', 'present.']
24 1
['present.', 'present.', 'not present.', 'not present.', 'present.', 'not present.', 'present.', 'present.']
27 1
['not present.', 'present.', 'present.', 'not present.', 'pres

In [ ]:
Subtask b results:
# 122/240 = 0.508 (Overall accuracy)
# 120/213 = 0.563 (Accuracy on TIN)
# 2/27 = 0.07 (Accuract on UNT)


In [19]:
test_dataloader = torch.utils.data.DataLoader(test_c_dataset['test'], batch_size=8)

with torch.no_grad():

  model.load_state_dict(torch.load(os.path.join('./olid_trained_models/', 't5_full_shot_epoch_4.pt')))

  model.eval()
  count_1 = 0
  count_2 = 0
  count_3 = 0

  for batch in tqdm.notebook.tqdm(test_dataloader):
    # when generating, we will use the logits of right-most token to predict the next token
    # so the padding should be on the left
    tokenizer = T5Tokenizer.from_pretrained('t5-small', cache_dir='./t5_cache')

    sentences = batch['tweet'] # use different length sentences to test batching
    for input in sentences:
      input = input + '<\s> ' + 'The offensive speech is targeted towards '
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)

    output_sequences = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=64,
        early_stopping=True,
        num_beams=10,
        num_return_sequences=1,
        no_repeat_ngram_size=2
    )
    preds = (tokenizer.batch_decode(output_sequences, skip_special_tokens=True,clean_up_tokenization_spaces=True))
    print (preds)
    for i,pred in enumerate(preds):
      if "individual." == pred and batch['label'][i]=='IND':
        count_1 = count_1 + 1
      if "group."==pred and (batch['label'][i]=='GRP'):
        count_2 = count_2 + 1
      if "neither."==pred and (batch['label'][i]=='OTH'):
        count_3 = count_3 + 1
    print(count_1, count_2, count_3)


  0%|          | 0/27 [00:00<?, ?it/s]

['not present.', 'not present.', 'present.', 'not present.', 'present.', 'not present.', 'present.', 'not present.']
0 0 0
['present.', 'present.', 'not present.', 'present.', 'not present.', 'present.', 'not present.', 'present.']
0 0 0
['present.', 'present.', 'not present.', 'not present.', 'present.', 'present.', 'present.', 'present.']
0 0 0
['not present.', 'not present.', 'present.', 'not present.', 'not present.', 'present.', 'not present.', 'present.']
0 0 0
['not present.', 'present.', 'not present.', 'not present.', 'not present.', 'present.', 'present.', 'not present.']
0 0 0
['not present.', 'not present.', 'present.', 'present.', 'not present.', 'not present.', 'not present.', 'not present.']
0 0 0
['not present.', 'present.', 'present.', 'present.', 'not present.', 'not present.', 'present.', 'not present.']
0 0 0
['present.', 'not present.', 'present.', 'present.', 'not present.', 'present.', 'not present.', 'not present.']
0 0 0
['present.', 'present.', 'present.', 'pr